In [1]:
# Dependencies and Setup
%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import time
import json
import re
import xml.etree.ElementTree as ET
import xmltodict
import collections
import itertools
from ratelimit import limits, sleep_and_retry
# Import API key


In [2]:
StateSelected = ['GA']

In [3]:
zipcodes=pd.read_csv('free-zipcode-database-Primary.csv')
zipcodes.head()

,Zipcode,State
0,705,PR
1,610,PR
2,611,PR
3,612,PR
4,601,PR


In [4]:
gazipcodes= zipcodes[zipcodes.State.isin(StateSelected)]

In [5]:
gazipcodes=gazipcodes.head(10)
len(gazipcodes)

10

## Perform API Calls

In [6]:

#conn = http.client.HTTPSConnection("search.onboard-apis.com") 

params = { 
    'accept': "application/json", 
    'apikey': "9d078487e223b1c4d54c3f3a3f628803", 
} 

In [7]:
FullAddress=[]
Street=[]
CityStateZip=[]
Latitude=[]
Longitude=[]
pd.Series(FullAddress)
pd.Series(Latitude)
pd.Series(Longitude)
Data_set=pd.DataFrame({'FullAddress':pd.Series(FullAddress),'Street':pd.Series(Street),'CityStateZip':pd.Series(CityStateZip),'Latitude':pd.Series(Latitude),'Longitude':pd.Series(Longitude)})
url=f"https://search.onboard-apis.com/propertyapi/v1.0.0/property/address?postalcode=postalcode&page=1&pagesize=100"
for index, row in gazipcodes.iterrows():
    postalcode=row['Zipcode']
    url=f"https://search.onboard-apis.com/propertyapi/v1.0.0/property/address?postalcode={postalcode}&page=1&pagesize=1000"
    res = requests.get(url,headers=params)
    data =json.loads(res.text)
    we=data
    try:
        d=pd.DataFrame(data['property'])
        d['FullAddress']=d['address'].apply(lambda a:a['oneLine'])
        d['Street']=d['address'].apply(lambda d:d['line1'])
        d['CityStateZip']=d['address'].apply(lambda d:d['line2'])
        d['Latitude']=d['location'].apply(lambda c:c['latitude'])
        d['Longitude']=d['location'].apply(lambda b:b['longitude'])
        datatoappend=d[['FullAddress','Street','CityStateZip','Latitude','Longitude']]
        Data_set=Data_set.append(datatoappend)
        
    
            
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
    


In [8]:
Data_set.head()

,FullAddress,Street,CityStateZip,Latitude,Longitude
0,"109 TAYLOR BRIDGE RD SW, CALHOUN, GA 30701",109 TAYLOR BRIDGE RD SW,"CALHOUN, GA 30701",34.417930,-84.937120
1,"137 TAYLOR BRIDGE RD SW # A, CALHOUN, GA 30701",137 TAYLOR BRIDGE RD SW # A,"CALHOUN, GA 30701",34.418477,-84.937531
2,"137 TAYLOR BRIDGE RD SW, CALHOUN, GA 30701",137 TAYLOR BRIDGE RD SW,"CALHOUN, GA 30701",34.418477,-84.937531
3,"314 ROBBINS CREEK TRL SW, CALHOUN, GA 30701",314 ROBBINS CREEK TRL SW,"CALHOUN, GA 30701",34.420551,-84.983000
4,"310 ROBBINS CREEK TRL SW, CALHOUN, GA 30701",310 ROBBINS CREEK TRL SW,"CALHOUN, GA 30701",34.420560,-84.983002


In [9]:
Data_set['Zilliw format add']=Data_set['Street'].replace(" ", "+",regex=True)
Data_set['Zilliw format Zipciy']=Data_set['CityStateZip'].replace(" ", "+",regex=True)


In [11]:
# Data_set['Zestimate']=" "
# Data_set['LastSoldPrice']=" "
# Data_set['Year Built']=" "
# Data_set['totalRooms']=" "
# Data_set['finishedSqFt']=" "
# Data_set['useCode']=" "
# Data_set['zpid']=" "
DataSet=Data_set.head(100)


In [19]:
#Next is to feed the addresess to zillow api
# OpenWeatherMap API Key
zid = 'X1-ZWz1go72alddzf_8469v'
#api_key = api_keys.api_key

# Starting URL for Weather Map API Call
#url = "http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=" + api_key 

#url = f"http://www.zillow.com/webservice/GetDeepSearchResults.htm?zws-id={api_key}&address=109 TAYLOR BRIDGE RD SW&citystatezip=CALHOUN GA 30701"  
#url = f"http://www.zillow.com/webservice/GetDeepSearchResults.htm?zws-id={api_key}&address=5888+Colonnade+Drive&citystatezip=Rex,+GA+30273"  


In [20]:
# response =requests.get(url)

# w=response.content
             



In [21]:
zpid=[]
zest=[]
addr =[]
city=[]
zipC=[]
lat =[]
lon =[]
type=[]
rooms=[]
sqft=[]
state=[]
lastsoldprice=[]
year_built=[]
c_value=[]

for index, row in Data_set.iterrows():
    add=row['Zilliw format add']
    statezip=row['Zilliw format Zipciy']
    params2=f'{add}&citystatezip={statezip}'
    url2 = f"http://www.zillow.com/webservice/GetDeepSearchResults.htm?zws-id={zid}&address={params2}"

   
    response2 = requests.get(url2, params=params2)
 
    we=response2.content
   
    tree=ET.fromstring(we)
    results=tree.findall('./response/results/result')
    
    if len(results)!=0:
        re=results[0] 
        dic=xmltodict.parse(ET.tostring(re))
        try: 
            
             Zid=dic['result'].get('zpid',np.nan)
             zestimate=dic['result']['zestimate']['amount'].get('#text',np.nan)
             strt=dic['result']['address'].get('street',np.nan)
             zip=dic['result']['address'].get('zipcode',np.nan)
             cit=dic['result']['address'].get('city',np.nan)
             price=dic['result'].get('lastSoldPrice',np.nan)
             room=dic['result'].get('bedrooms',np.nan)
             usecode=dic['result'].get('useCode',np.nan)
             sqftt=dic['result'].get('finishedSqFt',np.nan)
             yearbuilt=dic['result'].get('yearBuilt',np.nan)
             contyvalue=dic['result'].get('taxAssessment',np.nan)
             year_built.append(yearbuilt)
             zest.append(zestimate)
             zpid.append(Zid)
             addr.append(strt)
             zipC.append(zip)
             city.append(cit)
             lastsoldprice.append(price)
             rooms.append(room)
             type.append(usecode)
             sqft.append(sqftt)
             c_value.append(contyvalue)
             zestimate=[]
             Zid=[]
             price=[]
             room=[]
             usecode=[]
             sqftt=[]
             cit=[]
             usecode=[]
             yearbuilt=[]
             contyvalue=[]
             strt=[]
             print(f'got it updating')
             
        except (KeyError, IndexError):
             print(skip)
        


got it updating
got it updating
got it updating
got it updating
got it updating
got it updating
got it updating
got it updating
got it updating
got it updating
got it updating
got it updating
got it updating
got it updating
got it updating
got it updating
got it updating
got it updating
got it updating
got it updating
got it updating
got it updating
got it updating
got it updating
got it updating
got it updating
got it updating
got it updating
got it updating
got it updating
got it updating
got it updating
got it updating
got it updating
got it updating
got it updating
got it updating
got it updating
got it updating
got it updating
got it updating
got it updating
got it updating
got it updating
got it updating
got it updating
got it updating
got it updating
got it updating
got it updating
got it updating
got it updating
got it updating
got it updating
got it updating
got it updating
got it updating
got it updating
got it updating
got it updating
got it updating
got it updating
got it u

KeyboardInterrupt: 

In [22]:
re

<Element 'result' at 0x0000000005171B88>

In [26]:
d=pd.DataFrame({'Zillow Id':pd.Series(zpid),'Zestimate':pd.Series(zest),'Street':pd.Series(addr),'City':pd.Series(city),'ZipCode':pd.Series(zipC),'lastSold':pd.Series(lastsoldprice),'Bedrooms':pd.Series(rooms),'typeofhome':pd.Series(type),'Squareft':pd.Series(sqft),'YearBuilt':pd.Series(year_built),'CountTax':pd.Series(c_value)})

In [27]:
df=d.drop_duplicates(['Zillow Id'])
df.head()

,Zillow Id,Zestimate,Street,City,ZipCode,lastSold,Bedrooms,typeofhome,Squareft,YearBuilt,CountTax
0,111797219,440935,137 Taylor Bridge Rd SW # A,Calhoun,30701,NaN,3,SingleFamily,2998,2000,459600.0
1,111796580,576846,314 Robbins Creek Trl SW,CALHOUN,30701,NaN,4,SingleFamily,4609,2000,579500.0
2,76450055,868378,310 Robbins Creek Trl SW,CALHOUN,30701,NaN,6,SingleFamily,7838,2002,723300.0
3,117582985,470127,301 Robbins Creek Trl SW,CALHOUN,30701,"{'@currency': 'USD', '#text': '95000'}",5,SingleFamily,4708,2009,628700.0
4,76449706,689524,295 Robbins Creek Trl SW,CALHOUN,30701,"{'@currency': 'USD', '#text': '70000'}",5,SingleFamily,6800,1999,712400.0


In [28]:
df['LastSold']=df['lastSold'].apply(lambda l: l.get('#text', np.nan) if isinstance(l, dict) else np.nan)
df.head(10)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


,Zillow Id,Zestimate,Street,City,ZipCode,lastSold,Bedrooms,typeofhome,Squareft,YearBuilt,CountTax,LastSold
0,111797219,440935,137 Taylor Bridge Rd SW # A,Calhoun,30701,NaN,3,SingleFamily,2998,2000,459600.0,NaN
1,111796580,576846,314 Robbins Creek Trl SW,CALHOUN,30701,NaN,4,SingleFamily,4609,2000,579500.0,NaN
2,76450055,868378,310 Robbins Creek Trl SW,CALHOUN,30701,NaN,6,SingleFamily,7838,2002,723300.0,NaN
3,117582985,470127,301 Robbins Creek Trl SW,CALHOUN,30701,"{'@currency': 'USD', '#text': '95000'}",5,SingleFamily,4708,2009,628700.0,95000
4,76449706,689524,295 Robbins Creek Trl SW,CALHOUN,30701,"{'@currency': 'USD', '#text': '70000'}",5,SingleFamily,6800,1999,712400.0,70000
5,76449907,552215,2412 Mcdaniel Station Rd SW,CALHOUN,30701,NaN,3,SingleFamily,1835,1969,736770.0,NaN
6,223809739,82827,2493 Mcdaniel Station Rd SW,Calhoun,30701,NaN,2,SingleFamily,1086,1975,84200.0,NaN
7,76449884,154845,1637 Miller Ferry Rd SW,CALHOUN,30701,NaN,3,SingleFamily,2088,1965,124550.0,NaN
8,76449898,107844,2517 Mcdaniel Station Rd SW,CALHOUN,30701,"{'@currency': 'USD', '#text': '93500'}",2,SingleFamily,1464,1969,86300.0,93500
9,111796505,104273,1653 Miller Ferry Rd SW,CALHOUN,30701,"{'@currency': 'USD', '#text': '19000'}",5,SingleFamily,2176,2005,48030.0,19000


In [29]:
df.head()

,Zillow Id,Zestimate,Street,City,ZipCode,lastSold,Bedrooms,typeofhome,Squareft,YearBuilt,CountTax,LastSold
0,111797219,440935,137 Taylor Bridge Rd SW # A,Calhoun,30701,NaN,3,SingleFamily,2998,2000,459600.0,NaN
1,111796580,576846,314 Robbins Creek Trl SW,CALHOUN,30701,NaN,4,SingleFamily,4609,2000,579500.0,NaN
2,76450055,868378,310 Robbins Creek Trl SW,CALHOUN,30701,NaN,6,SingleFamily,7838,2002,723300.0,NaN
3,117582985,470127,301 Robbins Creek Trl SW,CALHOUN,30701,"{'@currency': 'USD', '#text': '95000'}",5,SingleFamily,4708,2009,628700.0,95000
4,76449706,689524,295 Robbins Creek Trl SW,CALHOUN,30701,"{'@currency': 'USD', '#text': '70000'}",5,SingleFamily,6800,1999,712400.0,70000


In [ ]:
df.columns

In [ ]:
selected=['Zestimate'], ['Street'], ['City'], ['ZipCode'], ['lastSold'],
           ['Bedrooms'], ['typeofhome'], ['Squareft'], ['YearBuilt'], ['CountTax'],
           ['LastSold']

In [ ]:
df=df[selected]